In [1]:
import sklearn
import numpy as np
from scipy import io
import tensorflow as tf
import os
import time
import h5py
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
import math
# import pandas as pd
import glob, os

In [2]:
ALL = h5py.File('M_0_LPS_features.mat')
# ALL  = h5py.File('B_0_features.mat')
# ALL1 = h5py.File('M_0_features.mat')
# ALL2 = h5py.File('T_0_features.mat')
# # ALL3 = h5py.File('B_5_features.mat')
# # ALL4 = h5py.File('B_10_features.mat')

In [3]:
data =  np.transpose(ALL['trainData'].value, axes=(1,0))
# data = np.r_[data,np.transpose(ALL1['trainData'].value, axes=(1,0))]
# data = np.r_[data,np.transpose(ALL2['trainData'].value, axes=(1,0))]
# data = np.r_[data,np.transpose(ALL3['trainData'].value, axes=(1,0))]
# data = np.r_[data,np.transpose(ALL4['trainData'].value, axes=(1,0))]

#data = np.r_[data,data,data]
print("Data Shape ", data.shape)

('Data Shape ', (5671270, 258))


/usr/lib64/python2.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [4]:
nf           = 1 # number of frame for each image
(len_data,b) = (data.shape)
image_row    = nf*2
# image_col  = 29

In [6]:
inputdim = b-1;
x        = data[:, 0:inputdim]
y        = data[:, inputdim].astype(int)

In [7]:
len_x   = int(len_data/nf)
y1      = y.reshape(len_x,nf)
frame_y = y1[:,0:1]
frame_x = x.reshape(len_x, nf ,inputdim)

In [8]:
len_train = int(len_x*0.6)
len_train = int(len_train/nf)*nf

len_test  = int(len_x*0.4)
len_test  = int(len_test/nf)*nf

print('len_train:', len_train)
print('len_test:', len_test)

('len_train:', 3402762)
('len_test:', 2268508)


In [9]:
del data

In [10]:
idx = np.random.permutation(len_x)
frame_x,frame_y = frame_x[idx], frame_y[idx]

In [11]:
frame_x_train    = frame_x[0:len_train, :, :]
frame_y_train    = frame_y[0:len_train, ]
frame_x_test     = frame_x[len_train:len_x, :, :]
frame_y_test     = frame_y[len_train:len_x, ]

In [12]:
def one_hot_encode(idx, vals=2):
    '''
    For use to one-hot encode the 10- possible labels
    '''
    len_idx=len(idx)
    out = np.zeros((len_idx, vals))
    for i in range(len_idx):
        out[i,idx[i]] = 1
        
    return out

In [13]:
frame_y_train = one_hot_encode(frame_y_train,2)
frame_y_test  = one_hot_encode(frame_y_test,2)

In [14]:
class DataHelper():
    
    def __init__(self):
        self.i = 0
        self.k = 0
        
        #self.all_train_batches = [data_batch1,data_batch2,data_batch3,data_batch4,data_batch5]
        #self.test_batch = [test_batch]
        
        self.training_data = None
        self.training_labels = None
        
        self.test_data = None
        self.test_labels = None
    
    def set_up_data(self,two_ch_input_training,y_training,two_ch_input_test,y_test):
        
        print("Setting Up Training Data and Labels")
        
        self.training_data= frame_x_train 
        train_len = len(self.training_data)
        
        self.training_labels = frame_y_train
        
        print("Setting Up Test Images and Labels")
        
        self.test_data = frame_x_test
        test_len = len(self.test_data)
        
        self.test_labels = frame_y_test

        
    def next_batch(self, batch_size):
        x = self.training_data[self.i:self.i+batch_size]
        y = self.training_labels[self.i:self.i+batch_size]
        self.i = (self.i + batch_size) % len(self.training_data)
        return x, y
    
    def next_batch_test(self, batch_size):
        x_test = self.test_data[self.k:self.k+batch_size]
        y_test = self.test_labels[self.k:self.k+batch_size]
        #print(self.k)
        self.k = (self.k + batch_size) % len(self.test_data)
        
        return x_test, y_test

In [15]:
dh = DataHelper()
dh.set_up_data(frame_x_train,frame_y_train,frame_x_test,frame_y_test)

Setting Up Training Data and Labels
Setting Up Test Images and Labels


In [15]:
## INPUT PARAMETERS ##
# x = tf.placeholder(tf.float32,[None,nf,inputdim],"x-input")
# x_image = tf.reshape(x, [-1,nf,inputdim, 1], name="x-image")  
# y_true = tf.placeholder(tf.float32,[None,1], "y-true")

In [18]:
K = 128
L = 64
M = 32
N = 512
nClasses = 2
div = 10

# tf.reset_default_graph()

# with tf.name_scope("hyperparameters"):
learning_rate = tf.placeholder(tf.float32)
keep_prob = tf.placeholder(tf.float32,name = "keep_prob")

# with tf.name_scope("inputs"):
x = tf.placeholder(tf.float32,[None,nf,inputdim],"x-input")
x_image = tf.reshape(x, [-1,nf,inputdim, 1], name="x-image")  
y_true = tf.placeholder(tf.float32,[None,nClasses], "y-true")  

# with tf.name_scope("model"):
W1 = tf.Variable(tf.truncated_normal([5,5,1,K], stddev=0.05))
B1 = tf.Variable(tf.ones([K])/div)
Y1 = tf.nn.relu(tf.nn.conv2d(x_image, W1, strides=[1,2,2,1], padding='SAME') + B1)

W2 = tf.Variable(tf.truncated_normal([5,5,K,L], stddev=0.05))
B2 = tf.Variable(tf.ones([L])/div)
Y2 = tf.nn.relu(tf.nn.conv2d(Y1, W2, strides=[1,2,2,1], padding='SAME') + B2)

W3 = tf.Variable(tf.truncated_normal([5,5,L,M], stddev=0.05))
B3 = tf.Variable(tf.ones([M])/div)
Y3 = tf.nn.relu(tf.nn.conv2d(Y2, W3, strides=[1,2,2,1], padding='SAME') + B3)

YY = tf.reshape(Y3, shape=[-1, 1*33*M])   

W4 = tf.Variable(tf.truncated_normal([1*33*M, N], stddev=0.05))
B4 = tf.Variable(tf.ones([N])/div)
Yf = tf.nn.relu(tf.matmul(YY, W4) + B4)
Y4 = tf.nn.dropout(Yf, keep_prob)

W5 = tf.Variable(tf.truncated_normal([N, 2], stddev=0.05))
B5 = tf.Variable(tf.ones([2])/div)
Y  = tf.nn.softmax(tf.matmul(Y4, W5) + B5)
Y = tf.identity(Y, name="output_t")

# with tf.name_scope("loss"):
cross_entropy = -tf.reduce_sum(y_true*tf.log(tf.clip_by_value(Y,1e-10,1.0)))
is_correct = tf.equal(tf.argmax(Y,1), tf.argmax(y_true,1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

# with tf.name_scope("train"):
optimizer = tf.train.AdamOptimizer(learning_rate)
train_op  = optimizer.minimize(cross_entropy) 

In [19]:
batch_size=500
size_test= int(len_test/batch_size)

print(size_test)

size_train= int(len_train/batch_size)
print(size_train)

4537
6805


In [20]:
checkpoint_dir = "full_model_M0_LPS/"
tf.gfile.MakeDirs(checkpoint_dir)

learningRates = np.hstack((1e-3*np.ones(10),
                           1e-4*np.ones(6),
                           1e-5*np.ones(4)))
nEpochs = len(learningRates)


with tf.Session() as sess:
    saver = tf.train.Saver()
    tf.train.write_graph(sess.graph_def,
                         checkpoint_dir,
                         "graph.pbtxt",
                         True)
    sess.run(tf.global_variables_initializer())    
    
    for epoch in np.arange(nEpochs):
        train_batch_count = 0 
        train_batch_acc_total = 0
        for i in range(size_train):
            batch = dh.next_batch(batch_size)
            sess.run(train_op, feed_dict={x: batch[0], y_true: batch[1],learning_rate: learningRates[epoch],keep_prob: 0.7})# hold_prob: 0.5 
            
            
            if i%1000 == 0:
                print('Currently on training step {}'.format(i))


            # PRINT OUT A MESSAGE EVERY 100 STEPS
            if i%(size_train-1) == 0 and i!=0:
                print('Currently on step {} for Testing'.format(i))
                
                test_batch_acc_total = 0
                test_batch_count = 0
                

                for k in range (size_test):
                    batch_test = dh.next_batch_test(batch_size)
                    test_batch_acc_total += sess.run(accuracy,feed_dict={x:batch_test[0],y_true:batch_test[1],learning_rate: learningRates[epoch],keep_prob: 1.0}) #,hold_prob:1.0
                    test_batch_count += 1
                    train_batch_acc_total += sess.run(accuracy,feed_dict={x:batch[0],y_true:batch[1],learning_rate: learningRates[epoch],keep_prob: 1.0}) #,hold_prob:1.0
                    train_batch_count += 1


                    if k%1000 == 0:
                        print('Currently on testing step {}'.format(k))

                print('Epoch: {}'.format(epoch))
                print('Testing Accuracy: {}\n'.format(test_batch_acc_total/test_batch_count))
                print('Training Accuracy: {}\n'.format(train_batch_acc_total/train_batch_count))
                
                
        tf.gfile.MakeDirs(checkpoint_dir + '/model' + str(epoch))
        checkpoint_file = os.path.join(checkpoint_dir + '/model' + str(epoch), "model")
        saver.save(sess, checkpoint_file)
        print("**** SAVED MODEL ****")      
        print("**** COMPLETED EPOCH ****")


Currently on training step 0
Currently on training step 1000
Currently on training step 2000
Currently on training step 3000
Currently on training step 4000
Currently on training step 5000
Currently on training step 6000
Currently on step 6804 for Testing
Currently on testing step 0
Currently on testing step 1000
Currently on testing step 2000
Currently on testing step 3000
Currently on testing step 4000
Epoch: 0
Testing Accuracy: 0.996841162279

Training Accuracy: 0.994000054415

**** SAVED MODEL ****
**** COMPLETED EPOCH ****
Currently on training step 0
Currently on training step 1000
Currently on training step 2000
Currently on training step 3000
Currently on training step 4000
Currently on training step 5000
Currently on training step 6000
Currently on step 6804 for Testing
Currently on testing step 0
Currently on testing step 1000
Currently on testing step 2000
Currently on testing step 3000
Currently on testing step 4000
Epoch: 1
Testing Accuracy: 0.996917425102

Training Accura

Currently on training step 6000
Currently on step 6804 for Testing
Currently on testing step 0
Currently on testing step 1000
Currently on testing step 2000
Currently on testing step 3000
Currently on testing step 4000
Epoch: 15
Testing Accuracy: 0.99928421523

Training Accuracy: 0.998000099923

**** SAVED MODEL ****
**** COMPLETED EPOCH ****
Currently on training step 0
Currently on training step 1000
Currently on training step 2000
Currently on training step 3000
Currently on training step 4000
Currently on training step 5000
Currently on training step 6000
Currently on step 6804 for Testing
Currently on testing step 0
Currently on testing step 1000
Currently on testing step 2000
Currently on testing step 3000
Currently on testing step 4000
Epoch: 16
Testing Accuracy: 0.999391336312

Training Accuracy: 1.00000011921

**** SAVED MODEL ****
**** COMPLETED EPOCH ****
Currently on training step 0
Currently on training step 1000
Currently on training step 2000
Currently on training step 3